In [1]:
import urbans

In [111]:
"""
TODO: Terminals for participles
V VT VsrT VMod VsrMod VPassOrgT VPassOrgMod

""" 
src_grammar = """S -> NPQ VP 
S -> NPQ was Nominal 
S -> NPQ did NPVMod 
S -> was NominalSub Vobl 
S -> NPQ Vobl 
S -> was NominalSub Adj 
S -> was NominalSub Nominal 
S -> did NominalSub VP 

NPVMod -> Nominal VMod
NPVMod -> Nominal VPrepOrgMod
NPVT -> Nominal VT
NPVT -> Nominal VPrepOrgT

VP -> V Nominal
VP -> was Vobl
VPT -> VT Nominal
VPT -> was VoblT 
VPMod -> VMod Nominal
VPMod -> was VoblMod

VPrep -> was VPrep
VPrep -> Vsr by
VPrepT -> was VPrepT
VPrepT -> VsrT by
VPrepMod -> was VPrepMod
VPrepMod -> VsrMod by

VPrepOrgMod -> was VPrepOrgMod
VPrepOrgMod -> VPassOrgMod by
VPrepOrgT -> was VPrepOrgT
VPrepOrgT -> VPassOrgT by

Vobl -> VPrep Nominal
VoblT -> VPrepT Nominal 
VoblMod -> VPrepMod Nominal

NPQ -> WhW Nominal 
NPQ -> WhW role caseO

commonNoun -> commonNoun RC

RC -> VoblMod
RC -> R VPMod
RC -> R NPVMod
RC -> whose role VPMod

VP -> VPT andVP
VP -> VPx andVP
VPx -> VPT punctVPT 
VPx -> VPx punctVPT 
andVP -> conj VP 
andVP -> punct conj VP
punctVPT -> punct VPT

VPMod -> VPT andVPMod
VPMod -> VPx andVPMod
VPx -> VPT punctVPT 
VPx -> VPx punctVPT 
andVPMod -> conj VPMod
andVPMod -> punct conj VPMod
punctVPT -> punct VPT

Vobl -> VoblT andVobl
Vobl -> Voblx andVobl 
Voblx -> VoblT punctVoblT 
Voblx -> Voblx punctVoblT 
andVobl -> conj Vobl 
andVobl -> punct conj Vobl
punctVoblT -> punct VoblT

VoblMod -> VoblT andVoblMod
VoblMod -> Voblx andVoblMod 
Voblx -> VoblT punctVoblT 
Voblx -> Voblx punctVoblT 
andVoblMod -> conj VoblMod 
andVoblMod -> punct conj VoblMod
punctVoblT -> punct VoblT

VPrep -> VPrepT andVPrep 
VPrep -> VPrepX andVPrep 
VPrepX -> VPrepT punctVPrepT 
VPrepX -> VPrepX punctVPrepT 
andVPrep -> conj VPrep 
andVPrep -> punct conj VPrep 
punctVPrepT -> punct VPrepT

VPrepMod -> VPrepT andVPrepMod 
VPrepMod -> VPrepX andVPrepMod 
VPrepX -> VPrepT punctVPrepT 
VPrepX -> VPrepX punctVPrepT 
andVPrepMod -> conj VPrepMod 
andVPrepMod -> punct conj VPrepMod 
punctVPrepT -> punct VPrepT

VPrepOrgMod -> VPrepOrgT andVPrepOrgMod 
VPrepOrgMod -> VPrepOrgX andVPrepOrgMod 
VPrepOrgX -> VPrepOrgT punctVPrepOrgT 
VPrepOrgX -> VPrepOrgX punctVPrepOrgT 
andVPrepOrgMod -> conj VPrepOrgMod 
andVPrepOrgMod -> punct conj VPrepOrgMod 
punctVPrepOrgT -> punct VPrepOrgT

V -> VT andV
V -> Vx andV
Vx -> VT punctVT
Vx -> Vx punctVT 
andV -> conj V
andV -> punct conj V 
punctVT -> punct VT

VT -> VT andVT
VT -> Vx andVT
Vx -> VT punctVT
Vx -> Vx punctVT 
andVT -> conj VT
andVT -> punct conj VT 
punctVT -> punct VT

VMod -> VT andVMod
VMod -> Vx andVMod
andVMod -> conj VMod
andVMod -> punct conj VMod

Vsr -> VsrT andVsr
Vsr -> Vsrx andVsr
Vsrx -> VsrT punctVsrT
Vsrx -> Vsrx punctVsrT 
andVsr -> conj Vsr
andVsr -> punct conj Vsr 
punctVsrT -> punct VsrT

VsrT -> VsrT andVsrT
VsrT -> Vsrx andVsrT
andVsrT -> conj VsrT
andVsrT -> punct conj VsrT 
punctVsrT -> punct VsrT

VsrMod -> VsrT andVsrMod
VsrMod -> Vsrx andVsrMod
andVsrMod -> conj VsrMod
andVsrMod -> punct conj VsrMod

Vx -> Vx punctVPrepT
Vx -> VT punctVPrepT
V -> Vx andVPrep
V -> VT andVPrep
VPrep -> VPrepT andV
VPrep -> VPrepX andV 
VPrepX -> VPrepT punctVT 
VPrepX -> VPrepX punctVT


VPrepOrgMod -> VPrepOrgT andVMod
VPrepOrgMod -> VPrepOrgX andVMod
VPrepOrgX -> VPrepOrgT punctVT
VPrepOrgX -> VPrepOrgX punctVT

VPassOrgMod -> VPassOrgT andVPassOrgMod
VPassOrgMod -> VPassOrgx andVPassOrgMod
VPassOrgx -> VPassOrgT punctVPassOrgT
VPassOrgx -> VPassOrgx punctVPassOrgT 
andVPassOrgMod -> conj VPassOrgMod
andVPassOrgMod -> punct conj VPassOrgMod 
punctVPassOrgT -> punct VPassOrgT


VMod -> Vx andVPrepMod
VMod -> VT andVPrepMod
VPrepMod -> VPrepT andVMod
VPrepMod -> VPrepX andVMod

NPVMod -> NPVT andNPVMod
NPVMod -> NPVx andNPVMod 
NPVx -> NPVT punctNPVT
NPVx -> NPVx punctNPVT 
andNPVMod -> conj NPVMod 
andNPVMod -> punct conj NPVMod 
punctNPVT -> punct NPVT

V -> F V
VT -> F VT
VMod -> F VMod
Vsr -> F Vsr
VsrT -> F VsrT
VsrMod -> F VsrMod
VPassOrgMod -> F VPassOrgMod
VPassOrgT -> F VPassOrgT

Nominal -> Name 
Nominal -> DP
Nominal -> commonNoun

NominalSub -> Name
NominalSub -> DPSub
NominalSub -> commonNoun

DP -> caseS role 
DPSub -> caseSSub role

caseS -> DP pS 
caseS -> Name pS 
caseSSub -> DPSub pS
caseSSub -> Name pS

DP -> det role caseO 
DPSub -> det role caseOSub

caseO -> of DP 
caseO -> of Name
caseOSub -> of DPSub
caseOSub -> of Name

DP -> det commonNoun
DPSub -> detSub commonNoun

Name -> Name andName 
Name -> Namex andName 
Namex -> Name punctName 
Namex -> Namex punctName 
andName -> conjN Name 
andName -> punctN conjN Name 
punctName -> punctN Name

commonNoun -> commonNoun andCommonNoun
commonNoun -> commonNounx andCommonNoun
commonNounx -> commonNoun punctCommonNoun
commonNounx -> commonNounx punctCommonNoun
andCommonNoun -> conjN commonNoun 
andCommonNoun -> punctN conjN commonNoun 
punctCommonNoun -> punctN commonNoun

role -> role androle
role -> rolex androle 
rolex -> role punctrole 
rolex -> rolex punctrole 
androle -> conjN role 
androle -> punctN conjN role
punctrole -> punctN role

commonNoun -> F commonNoun 
role -> F role
role -> Cnt of nat
commonNoun -> P commonNoun

commonNoun -> AdjNa commonNoun 
role -> AdjNa role







punct -> ","
punctN -> ","

Cnt -> 'country'
nat -> 'nationality'
P -> 'production'
F -> 'film' | 'art' | 'executive' | 'costume'

V -> 'direct' | 'produce' | 'edit' | 'married' | 'produced' | 'starred' | 'employ' | 'written' | 'edited' | 'marry' | 'distribute' | 'played' | 'influenced' | 'employed' | 'acquire' | 'acquired' | 'produce' | 'directed' | 'wrote' | 'influence' | 'found' | 'play' | 'star' | 'founded' | 'write' | 'direct' | 'distributed'
VT -> 'direct' | 'produce' | 'edit' | 'married' | 'produced' | 'starred' | 'employ' | 'written' | 'edited' | 'marry' | 'distribute' | 'played' | 'influenced' | 'employed' | 'acquire' | 'acquired' | 'produce' | 'directed' | 'wrote' | 'influence' | 'found' | 'play' | 'star' | 'founded' | 'write' | 'direct' | 'distributed'
Vsr -> 'direct' | 'produce' | 'edit' | 'married' | 'produced' | 'starred' | 'employ' | 'written' | 'edited' | 'marry' | 'distribute' | 'played' | 'influenced' | 'employed' | 'acquire' | 'acquired' | 'produce' | 'directed' | 'wrote' | 'influence' | 'found' | 'play' | 'star' | 'founded' | 'write' | 'direct' | 'distributed'
VsrT -> 'direct' | 'produce' | 'edit' | 'married' | 'produced' | 'starred' | 'employ' | 'written' | 'edited' | 'marry' | 'distribute' | 'played' | 'influenced' | 'employed' | 'acquire' | 'acquired' | 'produce' | 'directed' | 'wrote' | 'influence' | 'found' | 'play' | 'star' | 'founded' | 'write' | 'direct' | 'distributed'
VMod -> 'direct' | 'produce' | 'edit' | 'married' | 'produced' | 'starred' | 'employ' | 'written' | 'edited' | 'marry' | 'distribute' | 'played' | 'influenced' | 'employed' | 'acquire' | 'acquired' | 'produce' | 'directed' | 'wrote' | 'influence' | 'found' | 'play' | 'star' | 'founded' | 'write' | 'direct' | 'distributed'
VsrMod -> 'direct' | 'produce' | 'edit' | 'married' | 'produced' | 'starred' | 'employ' | 'written' | 'edited' | 'marry' | 'distribute' | 'played' | 'influenced' | 'employed' | 'acquire' | 'acquired' | 'produce' | 'directed' | 'wrote' | 'influence' | 'found' | 'play' | 'star' | 'founded' | 'write' | 'direct' | 'distributed'
VPassOrgT -> 'direct' | 'produce' | 'edit' | 'married' | 'produced' | 'starred' | 'employ' | 'written' | 'edited' | 'marry' | 'distribute' | 'played' | 'influenced' | 'employed' | 'acquire' | 'acquired' | 'produce' | 'directed' | 'wrote' | 'influence' | 'found' | 'play' | 'star' | 'founded' | 'write' | 'direct' | 'distributed'
VPassOrgMod -> 'direct' | 'produce' | 'edit' | 'married' | 'produced' | 'starred' | 'employ' | 'written' | 'edited' | 'marry' | 'distribute' | 'played' | 'influenced' | 'employed' | 'acquire' | 'acquired' | 'produce' | 'directed' | 'wrote' | 'influence' | 'found' | 'play' | 'star' | 'founded' | 'write' | 'direct' | 'distributed'

Name -> 'M0' | 'M1' | 'M2' | 'M3' | 'M4' | 'M5' | 'M6' | 'M7' | 'M8' | 'M9'

commonNoun -> 'character' | 'person' | 'composer' | 'prequel' | 'director' | 'writer' | 'company' | 'actor' | 'designer' | 'founder' | 'sequel' | 'producer' | 'spouse' | 'child' | 'editor' | 'employer' | 'employee' | 'distributor' | 'sibling' | 'star' | 'parent' | 'cinematographer' | 'screenwriter' | 'film'

role -> 'character' | 'person' | 'composer' | 'prequel' | 'director' | 'writer' | 'company' | 'actor' | 'designer' | 'founder' | 'sequel' | 'producer' | 'spouse' | 'child' | 'editor' | 'employer' | 'employee' | 'distributor' | 'sibling' | 'star' | 'parent' | 'cinematographer' | 'screenwriter' | 'film'
NPQ -> 'who' | 'what' | 'Who' | 'What'
WhW -> 'What' | 'Which' | 'what' | 'which'
did -> 'did' | 'Did'

conj -> 'and'
conjN -> 'and'

pS -> "'s"
of -> 'of'

det -> 'a' | 'an'
detSub -> 'a' | 'an'

by -> 'by'
Adj -> 'female' | 'American' | 'French' | 'Italian' | 'male' | 'Swedish' | 'Canadian' | 'British' | 'Spanish' | 'Mexican' | 'Chinese' | 'German' | 'American' | 'Dutch' | 'Japanese'
AdjNa -> 'female' | 'American' | 'French' | 'Italian' | 'male' | 'Swedish' | 'Canadian' | 'British' | 'Spanish' | 'Mexican' | 'Chinese' | 'German' | 'American' | 'Dutch' | 'Japanese'
was -> 'was' | 'were' | 'Was' | 'Were'
R -> 'that'
whose -> 'whose'
Qmark -> '?'






"""

In [46]:
src_to_target_grammar = {
    "S -> NPQ VP":"S -> NPQ が VP か", 
    "S -> NPQ was Nominal":"S -> Nominal は NPQ でしたか", # Q
    "S -> NPQ did NPVmod":"S -> NPVmod のは NPQ でしたか", # what did A write and B edit? -> A ga kaite, B ga hennshushita no ha nann deshitaka
    "S -> was NominalSub Vobl":"S -> NominalSub は Vobl か",
    "S -> NPQ Vobl":"NPQ が Vobl か", # same Q
    "S -> was NominalSub Adj":"S -> NominalSub は Adj でしたか",
    "S -> was NominalSub Nominal":"S -> NominalSub は Nominal でしたか",
    "S -> did NominalSub VP":"S -> NominalSub は VP か",

    # TODO: Chain rules to add for VMod and VPrepOrgMod
    "NPVmod -> Nominal VMod":"NPVmod -> Nominal が VMod",
    # todo: this rule basically follows VP, VPrepOrg finally derives as V
    "NPVmod -> Nominal VPrepOrgMod":"NPVmod -> Nominal を VPrepOrgMod", # VPrepOrg: passive in EN, active in JP 


    "VP -> V Nominal":"VP -> Nominal を V", # have lunch -> hirugohan wo taberu
    "VP -> was Vobl":"VP -> Vobl", # was done by him -> kare ni sareta
    "VPT -> VT Nominal":"VPT -> Nominal を VT",
    "VPT -> was VoblT":"VPT -> VoblT",
    "VPMod -> VMod Nominal":"VPMod -> Nominal を VMod",
    "VPMod -> was VoblMod":"VPMod -> VoblMod",

    "VPrep -> was VPrep":"VPrep -> VPrep",
    "VPrep -> V by":"VPrep -> V",
    "VPrepT -> was VPrepT":"VPrepT -> VPrepT",
    "VPrepT -> VT by":"VPrepT -> VT",
    "VPrepMod -> was VPrepMod":"VPrepMod -> VPrepMod",
    "VPrepMod -> VMod by":"VPrepMod -> VMod",
 
    "VPrepOrgMod -> was VPrepOrgMod":"VPrepOrgMod -> VPrepOrgMod",
    "VPrepOrgMod -> VPassOrgMod by":"VPrepOrgMod -> VPassOrgMod",

    "Vobl -> VPrep Nominal":"Vobl -> Nominal に VPrep", # done by nominal -> nominal ni sareta
    "VoblT -> VPrepT Nominal":"VoblT -> Nominal に VPrepT",
    "VoblMod -> VPrepMod Nominal":"VoblMod -> Nominal に VPrepMod",
    
    "NPQ -> WhW Nominal":"NPQ -> WhW Nominal", # which city -> dono machi
    # TODO: caseO -> of ... : caseO ... no
    "NPQ -> WhW role caseO":"NPQ -> caseO WhW role", # which character of Alice -> alisu no dono kyarakuta

    "commonNoun -> commonNoun RC":"commonNoun -> RC commonNoun", # people whose role was... -> yaku ga... hito

    "RC -> VoblMod":"RC -> VoblMod", # people attacked by... -> ...ni kougekisareta hito 
    "RC -> R VPMod":"RC -> VPMod", # people that eat... -> ...wo taberu hito
    "RC -> R NPVmod":"RC -> NPVmod", # people that you like -> anata ga sukina hito
    "RC -> whose role VPMod":"RC -> role が VPMod", # todo: role GA -> ...

    "VP -> VPT andVP":"VP -> VPT andVP",
    "VP -> VPx andVP":"VP -> VPx andVP",
    "VPx -> VPT punctVPT":"VPx -> VPT punctVPT",
    "VPx -> VPx punctVPT":"VPx -> VPx punctVPT",
    "andVP -> conj VP":"andVP -> conj VP",
    "andVP -> punct conj VP":"andVP -> punct conj VP",
    "punctVPT -> punct VPT":"punctVPT -> punct VPT",

    # VPMod -> VPT andVPMod
    # VPMod -> VPx andVPMod
    # VPx -> VPT punctVPT 
    # VPx -> VPx punctVPT 
    # andVPMod -> conj VPMod
    # andVPMod -> punct conj VPMod
    # punctVPT -> punct VPT

    # Vobl -> VoblT andVobl
    # Vobl -> Voblx andVobl 
    # Voblx -> VoblT punctVoblT 
    # Voblx -> Voblx punctVoblT 
    # andVobl -> conj Vobl 
    # andVobl -> punct conj Vobl
    # punctVoblT -> punct VoblT

    # VoblMod -> VoblT andVoblMod
    # VoblMod -> Voblx andVoblMod 
    # Voblx -> VoblT punctVoblT 
    # Voblx -> Voblx punctVoblT 
    # andVoblMod -> conj VoblMod 
    # andVoblMod -> punct conj VoblMod
    # punctVoblT -> punct VoblT

    # VPrep -> VPrepT andVPrep 
    # VPrep -> VPrepX andVPrep 
    # VPrepX -> VPrepT punctVPrepT 
    # VPrepX -> VPrepX punctVPrepT 
    # andVPrep -> conj VPrep 
    # andVPrep -> punct conj VPrep 
    # punctVPrepT -> punct VPrepT

    # VPrepMod -> VPrepT andVPrepMod 
    # VPrepMod -> VPrepX andVPrepMod 
    # VPrepX -> VPrepT punctVPrepT 
    # VPrepX -> VPrepX punctVPrepT 
    # andVPrepMod -> conj VPrepMod 
    # andVPrepMod -> punct conj VPrepMod 
    # punctVPrepT -> punct VPrepT

    # VPrepOrgMod -> VPrepOrgT andVPrepOrgMod 
    # VPrepOrgMod -> VPrepOrgX andVPrepOrgMod 
    # VPrepOrgX -> VPrepOrgT punctVPrepOrgT 
    # VPrepOrgX -> VPrepOrgX punctVPrepOrgT 
    # andVPrepOrgMod -> conj VPrepOrgMod 
    # andVPrepOrgMod -> punct conj VPrepOrgMod 
    # punctVPrepOrgT -> punct VPrepOrgT

    # V -> VT andV
    # V -> Vx andV
    # Vx -> VT punctVT
    # Vx -> Vx punctVT 
    # andV -> conj V
    # andV -> punct conj V 
    # punctVT -> punct VT

    # VT -> VT andVT
    # VT -> Vx andVT
    # Vx -> VT punctVT
    # Vx -> Vx punctVT 
    # andVT -> conj VT
    # andVT -> punct conj VT 
    # punctVT -> punct VT

    "DP -> det role caseO":"DP -> caseO role",
    "DPSub -> det role caseOSub":"DPSub -> caseOSub role",
# of -> no
    "caseO -> of DP":"caseO -> DP of",
    "caseO -> of Name":"caseO -> Name of",
    "caseOSub -> of DPSub":"caseOSub -> DPSub of",
    "caseOSub -> of Name":"caseOSub -> Name of",
    
    "role -> Cnt of nat":"role -> nat Cnt" # country of nationality -> 国籍国

}

special cases: like -> ga suki

In [47]:
from urbans import Translator

In [112]:
word_parser = Translator(src_grammar=src_grammar, src_to_tgt_grammar=src_to_target_grammar, src_to_tgt_dictionary=dict())

In [49]:
with open('Questions.txt') as f:
    lines = f.readlines()


In [34]:
tag_word_set, failed_sentences, ambiguity_sentences = word_parser.parse_words(sentences=lines)

Word parsing completed! 128695 sentences failed. 9227 sentences occurred ambiguity.


In [117]:
# reparsing
tag_word_set_2, failed_sentences_2, ambiguity_sentences_2 = word_parser.parse_words(sentences=failed_sentences[30000:50000])

Word parsing completed! 0 sentences failed. 3763 sentences occurred ambiguity.


In [162]:
F = ['film', 'art', 'executive', 'costume', 'production']

In [164]:
filter_data = [x for x in list(ambiguity_sentences_2.keys()) if
              all(y  not in x for y in F)]
len(filter_data)
filter_data

["Was M3 's spouse a person that M1 and M2 were founded by and employed\n",
 'Was a male director and writer of M0 , M1 , and M2 employed by M3 and employed by M4\n',
 "Was a person that M1 , M2 , and M3 were edited by and starred M4 's composer\n",
 'Was a American employee and founder of M0 and M1 influenced by M2 and influenced by M3\n',
 'What American female founder of M0 was influenced by M1 and influenced by M2 and M3\n',
 'What American founder of M0 , M1 , and M2 was influenced by M3\n',
 'What American spouse of M5 was influenced by M0 , M1 , M2 , and M3\n',
 'What American spouse of M6 was influenced by M0 and M1 and influenced by M2 , M3 , and M4\n',
 'What British French spouse of M3 was influenced by M0\n',
 'What British employee of M0 , M1 , and M2 was influenced by M3\n',
 'What British founder and employee of M0 and M1 was influenced by M2\n',
 'What British founder of M0 was influenced by M1 and influenced by M2 , M3 , and M4\n',
 'What British sibling of M0 was empl

In [167]:
import nltk
analyze_sen = "Was M3 's spouse a person that M1 and M2 were founded by and employed\n"
for i in ambiguity_sentences_2[analyze_sen]:
    i.draw()
ambiguity_sentences_2[analyze_sen]

[Tree('S', [Tree('was', ['Was']), Tree('NominalSub', [Tree('DPSub', [Tree('caseSSub', [Tree('Name', ['M3']), Tree('pS', ["'s"])]), Tree('role', ['spouse'])])]), Tree('Nominal', [Tree('DP', [Tree('det', ['a']), Tree('commonNoun', [Tree('commonNoun', ['person']), Tree('RC', [Tree('R', ['that']), Tree('NPVMod', [Tree('Nominal', [Tree('Name', [Tree('Name', ['M1']), Tree('andName', [Tree('conjN', ['and']), Tree('Name', ['M2'])])])]), Tree('VPrepOrgMod', [Tree('VPrepOrgT', [Tree('was', ['were']), Tree('VPrepOrgT', [Tree('VPassOrgT', ['founded']), Tree('by', ['by'])])]), Tree('andVMod', [Tree('conj', ['and']), Tree('VMod', ['employed'])])])])])])])])]),
 Tree('S', [Tree('was', ['Was']), Tree('NominalSub', [Tree('DPSub', [Tree('caseSSub', [Tree('Name', ['M3']), Tree('pS', ["'s"])]), Tree('role', ['spouse'])])]), Tree('Nominal', [Tree('DP', [Tree('det', ['a']), Tree('commonNoun', [Tree('commonNoun', ['person']), Tree('RC', [Tree('R', ['that']), Tree('NPVMod', [Tree('Nominal', [Tree('Name', [Tre

In [152]:
ambiguity_sentences_2['Was M2 produced by a production company that distributed M1\n']

[Tree('S', [Tree('was', ['Was']), Tree('NominalSub', [Tree('Name', ['M2'])]), Tree('Vobl', [Tree('VPrep', [Tree('Vsr', ['produced']), Tree('by', ['by'])]), Tree('Nominal', [Tree('DP', [Tree('det', ['a']), Tree('commonNoun', [Tree('commonNoun', [Tree('P', ['production']), Tree('commonNoun', ['company'])]), Tree('RC', [Tree('R', ['that']), Tree('VPMod', [Tree('VMod', ['distributed']), Tree('Nominal', [Tree('Name', ['M1'])])])])])])])])]),
 Tree('S', [Tree('was', ['Was']), Tree('NominalSub', [Tree('Name', ['M2'])]), Tree('Vobl', [Tree('VPrep', [Tree('Vsr', ['produced']), Tree('by', ['by'])]), Tree('Nominal', [Tree('DP', [Tree('det', ['a']), Tree('commonNoun', [Tree('P', ['production']), Tree('commonNoun', [Tree('commonNoun', ['company']), Tree('RC', [Tree('R', ['that']), Tree('VPMod', [Tree('VMod', ['distributed']), Tree('Nominal', [Tree('Name', ['M1'])])])])])])])])])])]

In [102]:
failed_sentences_2

['Did M2 employ a person that M1 was directed by , starred , and was edited by\n']

In [36]:
tag_word_set

{'did': {'Did', 'did'},
 'Name': {'M0', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8'},
 'V': {'acquire',
  'acquired',
  'direct',
  'directed',
  'distribute',
  'distributed',
  'edit',
  'edited',
  'employ',
  'employed',
  'found',
  'founded',
  'influence',
  'influenced',
  'married',
  'marry',
  'play',
  'played',
  'produce',
  'produced',
  'star',
  'starred',
  'write',
  'wrote'},
 'det': {'a'},
 'role': {'actor',
  'child',
  'cinematographer',
  'composer',
  'designer',
  'director',
  'distributor',
  'editor',
  'employee',
  'employer',
  'founder',
  'parent',
  'producer',
  'sibling',
  'spouse',
  'star',
  'writer'},
 'of': {'of'},
 'commonNoun': {'actor',
  'character',
  'cinematographer',
  'company',
  'composer',
  'designer',
  'director',
  'distributor',
  'editor',
  'person',
  'producer',
  'screenwriter'},
 'pS': {"'s"},
 'Adj': {'American',
  'British',
  'Canadian',
  'Chinese',
  'Dutch',
  'French',
  'German',
  'Italian',
  'Japanese',
  '